Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [19]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice, random
from copy import deepcopy

from tqdm.auto import tqdm
import numpy as np

In [20]:
#number of possible combinations of states
# 2*9*(8*(7*(6*(5*(4*(3*(2*(1+1)+1)+1)+1)+1)+1)+1)+1)
ALL_COMB = 2*(362880 + 362880 + 181440 + 60480 + 15120 + 3024 + 504 + 72 + 9)

ITERATIONS = 500_000

RANDOM_FIRST_TURN = False #Overwrites FIRST_TO_PLAY
FIRST_TO_PLAY = False

State = namedtuple('State', ['x', 'o'])

In [21]:
MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8]

In [22]:
def print_board(pos):
    """Nicely prints the board"""
    for r in range(3):
        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print('X', end='')
            elif MAGIC[i] in pos.o:
                print('O', end='')
            else:
                print('.', end='')
        print()
    print()

Updated state_value based on order of turns

In [23]:
def win(elements):
    """Checks is elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))

def state_value(turn, pos: State):
    """Evaluate state: +1 first player wins, +0,5 in case of draw if playing second"""
    if win(pos.x):
        return 1
    elif win(pos.o):
        return -1
    else:
        if turn:
            return 0
        else: 
            return 0.5

Trajectory functions

In [24]:
def random_game():
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    x_turn = choice([True, False])
    while available:
        cell = choice(list(available))
        if x_turn:
            state.x.add(cell)
        else:
            state.o.add(cell)
        x_turn =  not x_turn
        trajectory.append(deepcopy(state))
        available.remove(cell)
        if win(state.x) or win(state.o):
            break
    return x_turn, trajectory

def learning_game(value_dict, update_rate=1, first_to_play = True):
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    x_turn = bool(first_to_play)
    # exp based on chosen one but also on generated possible combination of moves, best case update_rate
    exp_rate = len(value_dict)/ALL_COMB * update_rate
    while available:
        cell = choice(list(available))
        if x_turn and exp_rate > random():
            max_value = -ITERATIONS
            for av_cell in available:
                hashable_state = (frozenset(state.x.copy().union({av_cell})), frozenset(state.o))
                #evaluate all possible moves
                if max_value < value_dict[hashable_state]:
                    max_value = value_dict[hashable_state]
                    cell = av_cell
        if x_turn:
            state.x.add(cell)
        else:
            state.o.add(cell)
        x_turn = not x_turn
        trajectory.append(deepcopy(state))
        available.remove(cell)
        if win(state.x) or win(state.o):
            break
    return trajectory


Applying the learning to the game

In [ ]:
def game(value_dict, first_to_play = True):
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    x_turn = bool(first_to_play)
    # exp based on chosen one but also on generated possible combination of moves, best case update_rate
    while available:
        cell = choice(list(available))
        if x_turn:
            max_value = -ITERATIONS
            for av_cell in available:
                hashable_state = (frozenset(state.x.copy().union({av_cell})), frozenset(state.o))
                #evaluate all possible moves
                if max_value < value_dict[hashable_state]:
                    max_value = value_dict[hashable_state]
                    cell = av_cell
        if x_turn:
            state.x.add(cell)
        else:
            state.o.add(cell)
        x_turn = not x_turn
        trajectory.append(deepcopy(state))
        available.remove(cell)
        if win(state.x) or win(state.o):
            break
    return trajectory

## Training

In [25]:
value_dictionary = defaultdict(float)
hit_state = defaultdict(int)
epsilon = 0.001
first = FIRST_TO_PLAY
#training
for steps in tqdm(range(ITERATIONS)):
    if RANDOM_FIRST_TURN: 
        first = choice([True, False])
    trajectory = learning_game(value_dictionary, 100, first)
    final_reward = state_value(first, trajectory[-1])
    #print("Final reward: ", final_reward)
    for state in trajectory:
        #print(state, end=": ")
        hashable_state = (frozenset(state.x), frozenset(state.o))
        hit_state[hashable_state] += 1
        #print(value_dictionary[hashable_state], end=",")
        value_dictionary[hashable_state] = value_dictionary[hashable_state] + epsilon * (final_reward - value_dictionary[hashable_state])
        #print(value_dictionary[hashable_state])



100%|██████████| 50000/50000 [00:07<00:00, 7063.64it/s]

The player was not the First to Play
Games_won: 87.69%
Games_draw: 11.40%
Games_lost: 0.91%


## Evaluation

In [ ]:
win_count = 0
draw_count = 0
lose_count = 0
for steps in tqdm(range(ITERATIONS)):
    if RANDOM_FIRST_TURN: 
        first = choice([True, False])
    trajectory = game(value_dictionary, first)
    final_reward = state_value(first, trajectory[-1])
    if final_reward > 0.5:
        win_count += 1
    elif final_reward < 0:
        lose_count += 1
    else :
        draw_count += 1
total_games = win_count+draw_count+ lose_count
if not RANDOM_FIRST_TURN:
    print(f"The player {"was" if FIRST_TO_PLAY else "was not"} the First to Play")
else:
    print("Turns were Randomized")
print(f"Games_won: {win_count/total_games:.2%}")
print(f"Games_draw: {draw_count/total_games:.2%}")
print(f"Games_lost: {lose_count/total_games:.2%}")

In [26]:
# filter(lambda e: len(e.x)+ len(e.o) > 5 ,value_dictionary.items())
sorted(value_dictionary.items(), key=lambda e: e[1], reverse=True)[0:10]

[((frozenset({5}), frozenset({7})), 0.36333923074662444),
 ((frozenset({5}), frozenset({9})), 0.3531043416953304),
 ((frozenset({5}), frozenset({4})), 0.26567752540382633),
 ((frozenset(), frozenset({7})), 0.24927147115187617),
 ((frozenset(), frozenset({9})), 0.24209722269717573),
 ((frozenset({5}), frozenset({8})), 0.24185137857390365),
 ((frozenset({5, 7, 8, 9}), frozenset({1, 2, 3, 4, 6})), 0.2340542374160329),
 ((frozenset({4}), frozenset({3})), 0.2309827057385861),
 ((frozenset({5}), frozenset({2})), 0.23097013226137186),
 ((frozenset({4, 5, 7, 8}), frozenset({1, 2, 3, 6, 9})), 0.23003295582574798)]

In [27]:
len(hit_state)

5477